In [15]:
import geopandas as gp
from shapely import wkt
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union
import pandas as pd
import numpy as np
from pprint import pprint
import os
import glob
import openpyxl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import plotly.express as px #if using plotly
import folium
import warnings
import uuid
from sklearn.preprocessing import StandardScaler

In [16]:
pd.set_option('display.max_columns', None)  # display all columns
pd.options.display.float_format = '{:20,.2f}'.format  # suppress scientific notation

### RQ #2) Which specific flares are the most impactful? 

### RQ #3) Which specific block grous are being most impacted? 


Exploratory question; no hypothesis testing

EJ index = (The Environmental Indicator Percentile for Block Group) x (Demographic Index for Block Group)
  
a) Determine flaring impact metric. 

Something like: buffer_population x EJ Index x flare volume  

b) Calculate impact per flare  

c) Visualize top_x flares by impact  


### Read in files

In [17]:
#ca_state = gp.read_file("data/CA_State_TIGER2016.shp")  # CA state
ca_counties = gp.read_file("data/CA_Counties_TIGER2016.shp")  # CA counties
ca_counties.rename(columns={'NAMELSAD':'cnty_name'}, inplace=True)  # old:new. Match col names for merging

In [18]:
#ca_bg = gp.read_file("data/ca_bg_clean.shp")  # CA block groups, cleaned
# CA block groups merged with EJscreen data
ca_bg_joined = gp.read_file("data/ca_bg_joined_clean.shp")

# update col name for correct area b/c it gets saved as a truncated version when saved as a shapefile out of the 
# main data cleaning notebook
ca_bg_joined.rename(columns={'shape_ar_1':'bg_area'}, inplace=True)  # old:new.
ca_bg_joined.rename(columns={'CNTY_NAME':'cnty_name'}, inplace=True)  # old:new.
print(f'BGs found: {len(ca_bg_joined)}')

BGs found: 25607


In [19]:
# just CA flares
ca_flares = gp.read_file("data/ca_flares_clean.shp")

# renaming 
ca_flares.rename(columns={'NAMELSAD':'cnty_name'}, inplace=True)  # old:new. Match col names for merging

print(f'Flares found: {len(ca_flares)}')

Flares found: 117


In [20]:
# set common crs for project
# epsg3310: https://epsg.io/3310-1739
# units: meters
meters_crs = 3310  # Projected crs. this should be good for this overlay() calculation and all of project. 

ca_counties = ca_counties.to_crs(meters_crs)
ca_flares = ca_flares.to_crs(meters_crs)
ca_bg_joined = ca_bg_joined.to_crs(meters_crs)

### Update ca_flares df

In [21]:
# set col list for BCM_avg calculation
bcm_list = ['BCM_2012','BCM_2013','BCM_2014','BCM_2015','BCM_2016','BCM_2017',
            'BCM_2018','BCM_2019','BCM_2020','BCM_2021']

# add new column for average BCM across all years
ca_flares['BCM_avg'] = ca_flares[bcm_list].mean(axis=1) 

In [22]:
# subset to only columns needed for Tableau
col_list = ['flare_id','flare_cate','cnty_name','BCM_2012','BCM_2013','BCM_2014','BCM_2015','BCM_2016','BCM_2017',
            'BCM_2018','BCM_2019','BCM_2020','BCM_2021','BCM_avg', 'geometry']

subset = ca_flares[col_list].copy()

# assign the subsetted copy back to the original variable name to keep total # of df names small 
# throughout project
ca_flares = subset

### Update ca_bg_joined df

In [23]:
# There are five BGs that seem to just be aquatic buffers around the actual county land.
# Dropping them for now 
# Explore if needed: ca_bg_joined.explore()

ids_to_drop = [60839900000, 61119901000, 60379902000, 60379903000, 60599901000]

# Drop the rows with those IDs
ca_bg_joined = ca_bg_joined[~ca_bg_joined['bg_id'].isin(ids_to_drop)]

In [24]:
# subset to only cols needed
subset = ca_bg_joined[['bg_id', 'cnty_name', 'ACSTOTPOP', 'MINORPOP',
                           'D_PM25_2', 'bg_area', 'geometry']].copy()

# assign the subsetted copy back to the original variable name
ca_bg_joined = subset

ca_bg_joined.rename(columns={'geometry':'bg_geom'}, inplace=True)  # old:new. Match col names for merging

## Create "baseline" df for use in further BG-specific and flare-specific analysis

In [25]:
# set geometry columns
ca_bg_joined = ca_bg_joined.set_geometry('bg_geom')

# and rename for clarity
ca_flares.rename(columns={'geometry':'flarepts_geom'}, inplace=True)  # old:new. Match col names for merging
ca_flares = ca_flares.set_geometry('flarepts_geom')

In [26]:
len(ca_bg_joined['bg_id'].unique())

25602

In [62]:
#buffer_size=100
#buffer_size=1000
buffer_size=2000

In [63]:
buffer_col = f"{buffer_size}_buff_geom"
ca_flares[buffer_col] = ca_flares['flarepts_geom'].buffer(distance=buffer_size)

ca_flares = ca_flares.set_geometry(buffer_col)

# subset of desired columns from flares_df
flares_sjoin_subset = ca_flares[[buffer_col, 'BCM_avg', 'flare_id', 'flare_cate']]

# Left join the flares_sjoin_subset cols to the bg_socialdf based on if any of the flares geometries intersect 
# with the bg geometries
intersect_baseline = gp.sjoin(ca_bg_joined, flares_sjoin_subset, how='left', predicate='intersects')

# Bring in buffer col that is not brought in by the sjoin() function for some unknown reason. Frustrating.
intersect_baseline = pd.merge(intersect_baseline, flares_sjoin_subset[['flare_id', buffer_col]], 
                             on='flare_id', how='left')

# set geom to the buffers and filter df down to just these BGs that have a flare_id associated with them. 
# There's no need to calculate any of the following for non-intersected BGs. 
intersect_baseline = intersect_baseline.set_geometry(buffer_col)
intersect_baseline = intersect_baseline[intersect_baseline.is_valid].copy()
print(f'{buffer_size}m BG-Buffer intersections: {len(intersect_baseline)}')

2000m BG-Buffer intersections: 501


In [64]:
# Iterate through each bg_id and flare_id pairing and calculate 
# the proportion of the buffer that intersects with the BG geom, relative to the total buffer size.
# We need to know the proportion of each buffer in each bg geom in order to assign the percentage of
# flare volume to each bg in the case of incomplete overlap.

# First check for null or invalid geometries in buffer_2000m column
intersect_baseline['buffer_valid'] = intersect_baseline[buffer_col].is_valid

# Initialize intersect proportion "b_int_prop" and intersect area "b_int_area" columns with NaN values
intersect_baseline[f'{buffer_size}_buff_int_prop'] = np.nan
intersect_baseline[f'{buffer_size}_buff_int_area'] = np.nan

# Loop through each block group and flare pairing
for index, row in intersect_baseline.iterrows():
    # Check if buffer_2000m geometry is valid
    if row['buffer_valid']:
        # Calculate intersection area
        intersection = row[buffer_col].intersection(row['bg_geom'])  # prop of buffer in the bg geom
        intersection_area = intersection.area
        # Calculate buffer area
        buffer_area = row[buffer_col].area
        # Calculate intersection proportion
        buffer_int_prop = intersection_area / buffer_area
        # Update int_prop and int_area columns with calculated values
        intersect_baseline.at[index, f'{buffer_size}_buff_int_prop'] = buffer_int_prop
        intersect_baseline.at[index, f'{buffer_size}_buff_int_area'] = intersection_area
        intersect_baseline.at[index, f'{buffer_size}_buff_tot_area'] = buffer_area

    else:
        # Assign 0 to b_int_prop and b_int_area if buffer_2000m geometry is invalid
        intersect_baseline.at[index, f'{buffer_size}_buff_int_prop'] = 0
        intersect_baseline.at[index, f'{buffer_size}_buff_int_area'] = 0
        intersect_baseline.at[index, f'{buffer_size}_buff_tot_area'] = 0


# Drop buffer_valid column as it's no longer needed
intersect_baseline.drop('buffer_valid', axis=1, inplace=True)

In [65]:
# Now a similar analysis but this time we want to know the BG-buffer intersection proportion relative to 
# the total area of the block group, in order to use that proportion to assign counts of BG demographic
# variables to the intersection area. 

# Combine all flares into a single multipolygon unit
temp = ca_flares.unary_union
all_buffers = gp.GeoDataFrame({'geometry': [temp]}, crs=ca_flares.crs) # convert back to geodf for processing

intersect_baseline = intersect_baseline.set_geometry('bg_geom')

# Make sure our geom column is set correctly to "bg_geom" rather than the flare buffer
if intersect_baseline.geometry.name == "bg_geom":
    int_temp = gp.overlay(intersect_baseline, all_buffers, how='intersection')  

    int_temp.rename(columns={'geometry':f'{buffer_size}_bg_int_geom'}, inplace=True)  # old:new. For clarity

    # Set geometry to the intersections
    int_temp = int_temp.set_geometry(f'{buffer_size}_bg_int_geom')  

    # Create new 'area' column for the areas of the intersections
    int_temp[f'{buffer_size}_bg_int_area'] = int_temp.area

    # Calculate the proportion of each block group intersection relative to total block broup area
    int_temp[f'{buffer_size}_bg_int_prop'] = int_temp[f'{buffer_size}_bg_int_area'] / int_temp['bg_area']

    # Merge the new area and props variables
    intersect_baseline = pd.merge(intersect_baseline, 
                                 int_temp[['bg_id',
                                           f'{buffer_size}_bg_int_prop',
                                           f'{buffer_size}_bg_int_area', 
                                           f'{buffer_size}_bg_int_geom']], 
                                 on=['bg_id'], 
                                 how='left').fillna({f'{buffer_size}_bg_int_prop':0, 
                                                     f'{buffer_size}_bg_int_area':0})

else:
    print("Error: Geometry column is not named 'bg_geom'")

In [66]:
# Apply the proportion to each demographic variable to find counts by variable
# i.e. this answers "how many of x live in the X buffer radius intersection area?"
demo_vars = ['ACSTOTPOP', 'MINORPOP']
for var in demo_vars:
    intersect_baseline[f'{buffer_size}_{var}_bg_int_count'] = (intersect_baseline[var] *
                                                               intersect_baseline[f'{buffer_size}_bg_int_prop'])

# Find *overall* proportions for each demo var by dividing var count by respective total population
# i.e. this answers "what overall percentage of this block group is minority status?"
intersect_baseline['MINORPOP_bg_totprop'] = intersect_baseline['MINORPOP'] / intersect_baseline['ACSTOTPOP']

# calculate percentage of minorities in intersection for each BG
intersect_baseline[f'{buffer_size}_bg_int_mnr_prop'] = (intersect_baseline[f'{buffer_size}_MINORPOP_bg_int_count']
                                                        / intersect_baseline[f'{buffer_size}_ACSTOTPOP_bg_int_count'])

# rename the flare categories for readability
intersect_baseline["flare_cate"] = (intersect_baseline["flare_cate"].
                                    replace({"flares_oil_downstream": "down_oil",
                                             "flares_upstream": "upstream"}))

In [67]:
# There are a ton of duplicate rows getting introduced at some point of the process. Ideally I'll come back and 
# troubleshoot, but in the meantime I'm just removing these dupes. 
df_baseline_all = intersect_baseline.drop_duplicates(subset=['bg_id', 'flare_id'])

In [68]:
len(df_baseline_all)

501

In [69]:
# grab two specific bgs to check
df_baseline_all[df_baseline_all['bg_id'].isin([60379800331, 60379800111])]


,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,bg_geom,index_right,BCM_avg,flare_id,flare_cate,2000_buff_geom,2000_buff_int_prop,2000_buff_int_area,2000_buff_tot_area,2000_bg_int_prop,2000_bg_int_area,2000_bg_int_geom,2000_ACSTOTPOP_bg_int_count,2000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,2000_bg_int_mnr_prop
72,60379800111,Los Angeles County,75,64,79.17,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",113.00,0.00,857477a0194312,down_oil,"POLYGON ((167197.511 -469609.028, 167187.881 -...",0.11,"1,376,354.57","12,546,193.96",1.00,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",75.00,64.00,0.85,0.85
77,60379800111,Los Angeles County,75,64,79.17,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",106.00,0.00,b30e8627682313,down_oil,"POLYGON ((166954.446 -469593.965, 166944.816 -...",0.11,"1,360,236.13","12,546,193.96",1.00,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",75.00,64.00,0.85,0.85
82,60379800111,Los Angeles County,75,64,79.17,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",107.00,0.00,caf4a94c957987,down_oil,"POLYGON ((167134.921 -469587.135, 167125.291 -...",0.11,"1,375,828.46","12,546,193.96",1.00,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",75.00,64.00,0.85,0.85
87,60379800111,Los Angeles County,75,64,79.17,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",108.00,0.00,f3188e83798361,down_oil,"POLYGON ((167283.806 -469521.668, 167274.175 -...",0.11,"1,376,685.28","12,546,193.96",1.00,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",75.00,64.00,0.85,0.85
92,60379800111,Los Angeles County,75,64,79.17,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",112.00,0.00,34208bde510417,down_oil,"POLYGON ((167197.510 -469510.426, 167187.879 -...",0.11,"1,376,685.28","12,546,193.96",1.00,"1,376,685.28","POLYGON ((164176.032 -467882.392, 164180.620 -...",75.00,64.00,0.85,0.85
474,60379800331,Los Angeles County,14,14,94.00,"28,637,875.64","POLYGON ((162259.677 -471740.729, 162351.365 -...",113.00,0.00,857477a0194312,down_oil,"POLYGON ((167197.511 -469609.028, 167187.881 -...",0.53,"6,627,666.43","12,546,193.96",0.24,"6,924,234.07","POLYGON ((163103.041 -470349.141, 163435.050 -...",3.39,3.39,1.00,1.00
479,60379800331,Los Angeles County,14,14,94.00,"28,637,875.64","POLYGON ((162259.677 -471740.729, 162351.365 -...",106.00,0.00,b30e8627682313,down_oil,"POLYGON ((166954.446 -469593.965, 166944.816 -...",0.54,"6,776,933.35","12,546,193.96",0.24,"6,924,234.07","POLYGON ((163103.041 -470349.141, 163435.050 -...",3.39,3.39,1.00,1.00
484,60379800331,Los Angeles County,14,14,94.00,"28,637,875.64","POLYGON ((162259.677 -471740.729, 162351.365 -...",107.00,0.00,caf4a94c957987,down_oil,"POLYGON ((167134.921 -469587.135, 167125.291 -...",0.53,"6,615,818.84","12,546,193.96",0.24,"6,924,234.07","POLYGON ((163103.041 -470349.141, 163435.050 -...",3.39,3.39,1.00,1.00
489,60379800331,Los Angeles County,14,14,94.00,"28,637,875.64","POLYGON ((162259.677 -471740.729, 162351.365 -...",108.00,0.00,f3188e83798361,down_oil,"POLYGON ((167283.806 -469521.668, 167274.175 -...",0.50,"6,266,837.20","12,546,193.96",0.24,"6,924,234.07","POLYGON ((163103.041 -470349.141, 163435.050 -...",3.39,3.39,1.00,1.00
494,60379800331,Los Angeles County,14,14,94.00,"28,637,875.64","POLYGON ((162259.677 -471740.729, 162351.365 -...",112.00,0.00,34208bde510417,down_oil,"POLYGON ((167197.510 -469510.426, 167187.879 -...",0.50,"6,316,848.90","12,546,193.96",0.24,"6,924,234.07","POLYGON ((163103.041 -470349.141, 163435.050 -...",3.39,3.39,1.00,1.00


# Aggregate data by BG and export files for Tableau

take baseline dfs and create copies clearly defined for this BG analysis, separate from the work that is needed
for the flare buffer analysis below. Both analyses modify these same two dfs in different ways and I want 
to make sure I keep the calculations separated

#### Standardize variables

EJ index and total population in the intersection(s) are already encoded at the level of a BG, so they're ready to normalize. 

Flare volume needs to multiplied by the proportion of the flare's buffer zone that resides in a BG, and then all (in the case of multiple flares in a BG) adjusted flare volumes summed up to generate a variable for total flare volume a BG is exposed to.  

In [339]:
def bg_calc_adj_flare_vol(df, buffer_size):
    """
    Function for calculating total flare volume to be assigned to each BG, based on amount of overlap.
    
    "bcm_adj" = adjusted bcm value for a given flare_id and bg_id, based on the proportion of the buffer zone that 
    overlaps with a given bg_id. Equal to the bcm_avg * b_int_prop (the overlap)
    
    "bcm_adj_tot" = the sum of the bcm_adj values for a given bg_id
    """
    
    # calculate the product of bcm_avg and buff_int_prop for each flare and bg combination
    df[f'{buffer_size}_bcm_int_adj'] = df['BCM_avg'] * df[f'{buffer_size}_buff_int_prop']
    
    # sum the adjusted volumes for all flare and bg combinations
    adj_vol = df.groupby(['flare_id', 'bg_id'])[f'{buffer_size}_bcm_int_adj'].sum().reset_index()
    
    # sum those adjusted volumes for all flares in each block group
    bg_vol = adj_vol.groupby('bg_id')[f'{buffer_size}_bcm_int_adj'].sum().reset_index()
    
    # merge the total adjusted block group volumes with the original dataframe
    df = pd.merge(df, bg_vol, on='bg_id', how='left', suffixes=('', '_tot'))
    
    return df

In [340]:
df_bg_intersect = df_baseline_all.groupby('bg_id').apply(bg_calc_adj_flare_vol, buffer_size=buffer_size)

In [341]:
# Define the variables to standardize

#'bcm_adj_tot' = Sum of the adjusted flare volume(s) that intersect w the BG. 
#'D_PM25_2' = EJ index for the BG
#'ACSTOTPOP_int_count' = BG population that resides in the intersection. = BGtotpop * intersection_proportion

varlist = [f'{buffer_size}_bcm_int_adj_tot', 'D_PM25_2', f'{buffer_size}_ACSTOTPOP_bg_int_count']

# Create a StandardScaler object that will transform selected variables to have a mean of zero and 
# sd of 1.
scaler = StandardScaler()

# Fit and transform the selected variables
intersect_norm = scaler.fit_transform(df_bg_intersect[varlist])

# Create new variables with the standardized values
intersect_norm = pd.DataFrame(intersect_norm, columns=[var + '_n' for var in varlist])

intersect_norm = intersect_norm.reset_index(drop=True)
df_bg_intersect = df_bg_intersect.reset_index(drop=True)

# Concatenate the new variables with the original DataFrame
df_bg_intersect = pd.concat([df_bg_intersect, intersect_norm], axis=1)

In [342]:
# Create calculations of # of counties affected and counts of BGs according to different
# "bins" of overlap

df_bg_intersect[f'{buffer_size}_bg_int_25%orless'] = (df_bg_intersect.groupby('bg_id')
                                        [f'{buffer_size}_bg_int_prop'].apply(lambda x: (x <= .25).sum()).sum())

df_bg_intersect[f'{buffer_size}_bg_int_50%orless'] = (df_bg_intersect.groupby('bg_id')
                                        [f'{buffer_size}_bg_int_prop'].apply(lambda x: (x <= .5).sum()).sum())

df_bg_intersect[f'{buffer_size}_bg_int_75%orless'] = (df_bg_intersect.groupby('bg_id')
                                        [f'{buffer_size}_bg_int_prop'].apply(lambda x: (x <= .75).sum()).sum())

df_bg_intersect[f'{buffer_size}_bg_int_anyint'] = (df_bg_intersect.groupby('bg_id')
                                        [f'{buffer_size}_bg_int_prop'].apply(lambda x: (x <= 1).sum()).sum())



In [343]:
# counties affected
temp = gp.GeoDataFrame(df_bg_intersect[['bg_id', 'flare_id', f'{buffer_size}_bg_int_geom']], 
                                         geometry=f'{buffer_size}_bg_int_geom')

counties_intersect = gp.sjoin(ca_counties, temp, how='inner', predicate='contains')  

df_bg_intersect[f'{buffer_size}_num_counties'] = len(counties_intersect['cnty_name'].unique())

In [344]:
df_bg_intersect.sample(10)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,bg_geom,index_right,BCM_avg,flare_id,flare_cate,2000_buff_geom,2000_buff_int_prop,2000_buff_int_area,2000_buff_tot_area,2000_bg_int_prop,2000_bg_int_area,2000_bg_int_geom,2000_ACSTOTPOP_bg_int_count,2000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,2000_bg_int_mnr_prop,2000_bcm_int_adj,2000_bcm_int_adj_tot,2000_bcm_int_adj_tot_n,D_PM25_2_n,2000_ACSTOTPOP_bg_int_count_n,2000_bg_int_25%orless,2000_bg_int_50%orless,2000_bg_int_75%orless,2000_bg_int_anyint,2000_num_counties
177,60375726002,Los Angeles County,1384,1347,65.51,"324,178.56","POLYGON ((165249.103 -465502.503, 165342.607 -...",111.00,0.00,90207aa4167979,down_oil,"POLYGON ((165836.943 -465162.056, 165827.312 -...",0.02,"295,732.38","12,546,193.96",0.91,"295,732.38","POLYGON ((165249.103 -465502.503, 165342.607 -...","1,262.56","1,228.80",0.97,0.97,0.00,0.00,-0.47,0.86,0.94,239,311,365,501,11
404,60971535013,Sonoma County,1096,179,9.31,"13,049,779.82","POLYGON ((-244627.886 39440.455, -244627.595 3...",114.00,0.00,1a1cbbe3166258,upstream,"POLYGON ((-237947.045 35040.908, -237956.675 3...",0.04,"443,456.18","12,546,193.96",0.03,"443,456.18","POLYGON ((-238842.224 36707.635, -238847.875 3...",37.24,6.08,0.16,0.16,0.00,0.00,-0.47,-1.50,-0.78,239,311,365,501,11
242,60375759023,Los Angeles County,1580,1259,57.09,"94,469.72","POLYGON ((166942.753 -469705.795, 166941.427 -...",112.00,0.00,34208bde510417,down_oil,"POLYGON ((167197.510 -469510.426, 167187.879 -...",0.01,"84,186.10","12,546,193.96",1.00,"94,469.72","POLYGON ((166942.753 -469705.795, 166941.427 -...","1,580.00","1,259.00",0.80,0.80,0.00,0.00,-0.47,0.51,1.38,239,311,365,501,11
12,60133141052,Contra Costa County,3516,3317,59.23,"616,116.21","POLYGON ((-171047.294 2917.300, -171030.554 29...",89.00,0.01,342ab9d4310483,upstream,"POLYGON ((-167588.809 339.717, -167598.440 143...",0.00,"3,828.89","12,546,193.96",0.01,"5,881.86","POLYGON ((-170363.356 2192.815, -170363.386 21...",33.57,31.67,0.94,0.94,0.00,0.00,-0.47,0.60,-0.78,239,311,365,501,11
380,60710071064,San Bernardino County,1191,821,43.47,"1,836,457.04","POLYGON ((245694.256 -440568.666, 245766.160 -...",115.00,0.00,7c149dcd266912,upstream,"POLYGON ((247587.514 -437618.082, 247577.883 -...",0.00,"24,519.68","12,546,193.96",0.01,"24,519.68","POLYGON ((246679.118 -439292.731, 246813.148 -...",15.90,10.96,0.69,0.69,0.00,0.00,-0.47,-0.07,-0.81,239,311,365,501,11
388,60710127001,San Bernardino County,2282,1738,52.62,"25,968,777.78","POLYGON ((222126.742 -436025.652, 222125.011 -...",116.00,0.00,ade49180235149,upstream,"POLYGON ((227403.428 -439194.020, 227393.798 -...",0.74,"9,308,870.24","12,546,193.96",0.36,"9,308,870.24","MULTIPOLYGON (((227377.431 -439500.387, 227364...",818.01,623.01,0.76,0.76,0.00,0.00,-0.46,0.32,0.31,239,311,365,501,11
454,61110076141,Ventura County,1881,785,18.11,"78,126,778.44","POLYGON ((99422.910 -408192.864, 99452.299 -40...",15.00,0.00,e4c4a3a5502407,upstream,"POLYGON ((106073.072 -404640.555, 106063.441 -...",0.10,"1,216,347.57","12,546,193.96",0.06,"4,415,820.09","MULTIPOLYGON (((109801.032 -404754.457, 109824...",106.32,44.37,0.42,0.42,0.00,0.00,-0.02,-1.13,-0.68,239,311,365,501,11
53,60190039003,Fresno County,1790,1601,53.76,"218,403,800.99","POLYGON ((-20303.099 -148116.834, -20300.228 -...",83.00,0.00,b7ba5a2f183790,upstream,"POLYGON ((-9949.702 -150361.388, -9959.333 -15...",1.00,"12,546,193.96","12,546,193.96",0.06,"12,546,193.96","POLYGON ((-9949.702 -150361.388, -9959.333 -15...",102.83,91.97,0.89,0.89,0.00,0.00,-0.42,0.37,-0.69,239,311,365,501,11
397,60830020061,Santa Barbara County,1542,505,4.55,"370,485,332.81","POLYGON ((-48200.248 -353073.213, -48118.835 -...",101.00,0.00,3a42d431295983,upstream,"POLYGON ((-34516.426 -355944.880, -34526.057 -...",1.00,"12,546,193.96","12,546,193.96",0.10,"37,746,676.82","MULTIPOLYGON (((-25798.027 -353596.302, -25807...",157.11,51.45,0.33,0.33,0.00,0.00,0.10,-1.70,-0.61,239,311,365,501,11
79,60290033042,Kern County,1040,279,31.94,"1,051,991,

In [345]:
# There are a ton of duplicate rows getting introduced at some point of the process. Ideally I'll come back and 
# troubleshoot, but in the meantime I'm just removing these dupes. 
df_bg_intersect = df_bg_intersect.drop_duplicates(subset=['bg_id', 'flare_id'])

In [346]:
len(df_bg_intersect)

501

In [347]:
# # check calculations against two BGs
# # Should see a single "bcm_adj_tot" value per BG, and different adjusted "bcm_adj" values per flare_id 
# #depending on the amount of overlap.

# # filter the dataframe and format specified columns to display 4 decimal places
# filtered_df = df_bg_intersect_all[df_bg_intersect_all['bg_id'].isin([60379800331, 60379800111])].copy()
# filtered_df[['BCM_avg', f'{buffer_size}_bcm_int_adj', f'{buffer_size}_bcm_int_adj_tot', 
#              f'{buffer_size}_bcm_int_adj_tot_n']] = filtered_df[['BCM_avg', f'{buffer_size}_bcm_int_adj',
#                                                                  f'{buffer_size}_bcm_int_adj_tot', 
#                                                                  f'{buffer_size}_bcm_int_adj_tot_n']].apply(lambda x: x.apply(lambda y: '{:.4f}'.format(y)))


In [348]:
# filtered_df

### Split geom data from rest of data b/c column names must be <10 characters for shapefiles and that makes naming my different columns unnecessarily confusing. I export a csv of everything but the geometry, export a shapefile of just the geometry, then join the two back together in Tableau. 

In [349]:
df_bg_intersect.sample(1)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,bg_geom,index_right,BCM_avg,flare_id,flare_cate,2000_buff_geom,2000_buff_int_prop,2000_buff_int_area,2000_buff_tot_area,2000_bg_int_prop,2000_bg_int_area,2000_bg_int_geom,2000_ACSTOTPOP_bg_int_count,2000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,2000_bg_int_mnr_prop,2000_bcm_int_adj,2000_bcm_int_adj_tot,2000_bcm_int_adj_tot_n,D_PM25_2_n,2000_ACSTOTPOP_bg_int_count_n,2000_bg_int_25%orless,2000_bg_int_50%orless,2000_bg_int_75%orless,2000_bg_int_anyint,2000_num_counties
138,60374024032,Los Angeles County,4721,3724,53.14,"1,776,570.77","POLYGON ((200491.627 -439671.323, 200498.298 -...",104.00,0.00,ac6392c6458336,upstream,"POLYGON ((203158.098 -439404.603, 203148.468 -...",0.11,"1,389,122.60","12,546,193.96",0.78,"1,389,122.60","POLYGON ((200491.627 -439671.323, 200498.298 -...","3,691.41","2,911.84",0.79,0.79,0.00,0.00,-0.47,0.34,4.34,239,311,365,501,11


### 100

In [258]:
df_bgintersect_geo100 = gp.GeoDataFrame(df_bg_intersect[['bg_id', 'flare_id', '100_bg_int_geom']], 
                                         geometry='100_bg_int_geom')

print(len(df_bgintersect_geo100[df_bgintersect_geo100.is_valid]))

df_bgintersect_100 = df_bg_intersect.drop(['bg_geom', f'{buffer_size}_buff_geom', f'{buffer_size}_bg_int_geom'], axis=1)

118


In [259]:
# Add in a buffer_size col for filtering in Tableau
df_bgintersect_geo100['buffer_size'] = 100

# and change the geom col name back to a general name so the 
# three buffer sizes can now be merged into one column
df_bgintersect_geo100.rename(columns={'100_bg_int_geom':'bg_int_geom'}, inplace=True)  # old:new. Match col names for merging

In [260]:
df_bgintersect_100.sample(2)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,index_right,BCM_avg,flare_id,flare_cate,100_buff_int_prop,100_buff_int_area,100_buff_tot_area,100_bg_int_prop,100_bg_int_area,100_ACSTOTPOP_bg_int_count,100_MINORPOP_bg_int_count,MINORPOP_bg_totprop,100_bg_int_mnr_prop,100_bcm_int_adj,100_bcm_int_adj_tot,100_bcm_int_adj_tot_n,D_PM25_2_n,100_ACSTOTPOP_bg_int_count_n,100_bg_int_25%orless,100_bg_int_50%orless,100_bg_int_75%orless,100_bg_int_anyint,100_num_counties
76,61110003021,Ventura County,1922,1570,31.02,"38,180,504.02",41.00,0.00,a02a1445648755,upstream,1.00,"31,365.48","31,365.48",0.00,"64,580.35",3.25,2.66,0.82,0.82,0.00,0.00,-0.65,-0.24,0.29,118,118,118,118,11
85,61110012041,Ventura County,2831,1652,29.14,"120,975,492.62",19.00,0.00,73d8f0e8112227,upstream,1.00,"31,365.48","31,365.48",0.00,"83,592.84",1.96,1.14,0.58,0.58,0.00,0.01,1.10,-0.32,0.03,118,118,118,118,11


In [261]:
df_bgintersect_geo100.sample(2)

,bg_id,flare_id,bg_int_geom,buffer_size
4,60133150001,139d4801336969,"MULTIPOLYGON (((-180371.821 3052.703, -180377....",100
14,60190082006,e7d48e6c798117,"MULTIPOLYGON (((-9884.386 -168129.034, -9884.8...",100


In [198]:
#df_bgintersect_geo100.to_file('data/df_bg_geo2000.shp', driver='ESRI Shapefile')

### 1000

In [281]:
df_bgintersect_geo1000 = gp.GeoDataFrame(df_bg_intersect[['bg_id', 'flare_id', '1000_bg_int_geom']], 
                                         geometry='1000_bg_int_geom')

print(len(df_bgintersect_geo1000[df_bgintersect_geo1000.is_valid]))

df_bgintersect_1000 = df_bg_intersect.drop(['bg_geom', f'{buffer_size}_buff_geom', 
                                            f'{buffer_size}_bg_int_geom'], axis=1)

# Add in a buffer_size col for filtering in Tableau
df_bgintersect_geo1000['buffer_size'] = 1000

# and change the geom col name back to a general name so the 
# three buffer sizes can now be merged into one column
df_bgintersect_geo1000.rename(columns={'1000_bg_int_geom':'bg_int_geom'}, inplace=True)  # old:new.

216


In [282]:
df_bgintersect_1000.head(2)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,index_right,BCM_avg,flare_id,flare_cate,1000_buff_int_prop,1000_buff_int_area,1000_buff_tot_area,1000_bg_int_prop,1000_bg_int_area,1000_ACSTOTPOP_bg_int_count,1000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,1000_bg_int_mnr_prop,1000_bcm_int_adj,1000_bcm_int_adj_tot,1000_bcm_int_adj_tot_n,D_PM25_2_n,1000_ACSTOTPOP_bg_int_count_n,1000_bg_int_25%orless,1000_bg_int_50%orless,1000_bg_int_75%orless,1000_bg_int_anyint,1000_num_counties
0,60133132032,Contra Costa County,2637,2125,47.88,"2,606,490.18",89.00,0.01,342ab9d4310483,upstream,0.13,"418,998.49","3,136,548.49",0.16,"422,883.57",427.83,344.77,0.81,0.81,0.00,0.00,-0.30,0.31,0.95,193,204,213,216,11
1,60133132032,Contra Costa County,2637,2125,47.88,"2,606,490.18",90.00,0.01,47291ae3406417,upstream,0.13,"422,191.34","3,136,548.49",0.16,"422,883.57",427.83,344.77,0.81,0.81,0.00,0.00,-0.30,0.31,0.95,193,204,213,216,11


In [32]:
#df_bgintersect_geo2000.to_file('data/df_bg_geo2000.shp', driver='ESRI Shapefile')

/Users/natebender/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.


### 2000

In [302]:
# Create a new geodataframe with just the id and geom columns
df_bgintersect_geo2000 = gp.GeoDataFrame(df_bg_intersect[['bg_id', 'flare_id', '2000_bg_int_geom']], 
                                         geometry='2000_bg_int_geom')

print(len(df_bgintersect_geo2000[df_bgintersect_geo2000.is_valid]))


# Subset the original dataframe and drop unnecessary columns
df_bgintersect_2000 = df_bg_intersect.drop(['bg_geom', f'{buffer_size}_buff_geom', 
                                         f'{buffer_size}_bg_int_geom'], axis=1)

# Add in a buffer_size col for filtering in Tableau
df_bgintersect_geo2000['buffer_size'] = 2000

# and change the geom col name back to a general name so the 
# three buffer sizes can now be merged into one column
df_bgintersect_geo2000.rename(columns={'2000_bg_int_geom':'bg_int_geom'}, inplace=True)  # old:new.

501


In [303]:
df_bgintersect_2000.head(2)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,index_right,BCM_avg,flare_id,flare_cate,2000_buff_int_prop,2000_buff_int_area,2000_buff_tot_area,2000_bg_int_prop,2000_bg_int_area,2000_ACSTOTPOP_bg_int_count,2000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,2000_bg_int_mnr_prop,2000_bcm_int_adj,2000_bcm_int_adj_tot,2000_bcm_int_adj_tot_n,D_PM25_2_n,2000_ACSTOTPOP_bg_int_count_n,2000_bg_int_25%orless,2000_bg_int_50%orless,2000_bg_int_75%orless,2000_bg_int_anyint,2000_num_counties
0,60133132031,Contra Costa County,729,495,33.27,"514,270.79",89.00,0.01,342ab9d4310483,upstream,0.04,"513,196.18","12,546,193.96",1.00,"513,196.18",727.48,493.97,0.68,0.68,0.00,0.00,-0.28,-0.50,0.19,239,311,365,501,11
1,60133132031,Contra Costa County,729,495,33.27,"514,270.79",90.00,0.01,47291ae3406417,upstream,0.04,"512,561.56","12,546,193.96",1.00,"513,196.18",727.48,493.97,0.68,0.68,0.00,0.00,-0.28,-0.50,0.19,239,311,365,501,11


In [51]:
#df_bgintersect_geo5000.to_file('data/df_bg_geo5000.shp', driver='ESRI Shapefile')

/Users/natebender/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.


### Combine

#### Social data

In [304]:
def filter_dataframe(df, prefix):
    cols_to_keep = ['bg_id', 'flare_id'] + [col for col in df.columns if col.startswith(prefix)]
    return df[cols_to_keep].copy()

In [305]:
# filter the 2nd and 3rd dataframes to only include columns starting with the buffer cols
# eg '5000_' plus the ID columns
# all the non "5000_" columns in these dfs are identical to the first df, so can be ignored.
# select columns that start with the appropriate prefix and keep 'bg_id' and 'flare_id'

df_bgintersect_100_filtered = filter_dataframe(df_bgintersect_100, '100_')
df_bgintersect_1000_filtered = filter_dataframe(df_bgintersect_1000, '1000_')

In [306]:
df_bg_intersect_temp = df_bgintersect_2000.merge(df_bgintersect_1000_filtered, 
                                               on=['bg_id', 'flare_id'], how='outer')


df_bg_intersect_all = df_bg_intersect_temp.merge(df_bgintersect_100_filtered, 
                                               on=['bg_id', 'flare_id'], how='outer')

In [308]:
len(df_bg_intersect_all)

501

In [309]:
df_bg_intersect_all[df_bg_intersect_all['bg_id'].isin([60372911201])]


,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,index_right,BCM_avg,flare_id,flare_cate,2000_buff_int_prop,2000_buff_int_area,2000_buff_tot_area,2000_bg_int_prop,2000_bg_int_area,2000_ACSTOTPOP_bg_int_count,2000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,2000_bg_int_mnr_prop,2000_bcm_int_adj,2000_bcm_int_adj_tot,2000_bcm_int_adj_tot_n,D_PM25_2_n,2000_ACSTOTPOP_bg_int_count_n,2000_bg_int_25%orless,2000_bg_int_50%orless,2000_bg_int_75%orless,2000_bg_int_anyint,2000_num_counties,1000_buff_int_prop,1000_buff_int_area,1000_buff_tot_area,1000_bg_int_prop,1000_bg_int_area,1000_ACSTOTPOP_bg_int_count,1000_MINORPOP_bg_int_count,1000_bg_int_mnr_prop,1000_bcm_int_adj,1000_bcm_int_adj_tot,1000_bcm_int_adj_tot_n,1000_ACSTOTPOP_bg_int_count_n,1000_bg_int_25%orless,1000_bg_int_50%orless,1000_bg_int_75%orless,1000_bg_int_anyint,1000_num_counties,100_buff_int_prop,100_buff_int_area,100_buff_tot_area,100_bg_int_prop,100_bg_int_area,100_ACSTOTPOP_bg_int_count,100_MINORPOP_bg_int_count,100_bg_int_mnr_prop,100_bcm_int_adj,100_bcm_int_adj_tot,100_bcm_int_adj_tot_n,100_ACSTOTPOP_bg_int_count_n,100_bg_int_25%orless,100_bg_int_50%orless,100_bg_int_75%orless,100_bg_int_anyint,100_num_counties
127,60372911201,Los Angeles County,2548,2513,67.53,"701,466.47",105.00,0.00,4338c869142492,upstream,0.06,"701,466.47","12,546,193.96",1.00,"701,466.47","2,548.00","2,513.00",0.99,0.99,0.00,0.00,-0.47,0.94,2.74,239,311,365,501,11,0.07,"210,114.49","3,136,548.49",0.30,"210,114.49",763.22,752.73,0.99,0.00,0.00,-0.73,2.12,193.00,204.00,213.00,216.00,11.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [372]:
len(df_bg_intersect_all)

501

In [310]:
df_bg_intersect_all.to_csv('data/df_bg_impact_all.tsv', sep='\t', index=False)

#### Geo data

In [85]:
df_bg_intersect_geoall = pd.concat([df_bgintersect_geo100, 
                                    df_bgintersect_geo1000,
                                    df_bgintersect_geo2000])

In [376]:
501-216-118

167

In [375]:
# num intersections at 2000m: 501
# at 1000m: 216
# at 100m: 118
# Should equal 835 total
len(df_bg_intersect_geoall)

835

In [87]:
df_bg_intersect_geoall.sample(10)

,bg_id,flare_id,bg_int_geom,buffer_size
445,61110028001,686255e8215591,"POLYGON ((71695.563 -419804.179, 71100.316 -41...",2000
15,60133141061,47291ae3406417,"POLYGON ((-169532.927 2341.461, -169533.073 23...",2000
189,60375754011,34208bde510417,"POLYGON ((166205.027 -467864.628, 166221.671 -...",2000
161,60375411003,4338c869142492,"POLYGON ((160765.897 -455515.932, 160827.147 -...",2000
300,60376502004,36c62e67595483,"POLYGON ((154238.777 -460276.155, 154288.649 -...",2000
473,61110095004,c2a58b7a749801,"MULTIPOLYGON (((63306.202 -412709.885, 63202.6...",2000
1,60133132042,47291ae3406417,"POLYGON ((-169488.809 339.717, -169489.291 329...",100
166,61110012041,54a0381d920615,"POLYGON ((66563.741 -411899.842, 66661.238 -41...",1000
35,60133552011,342ab9d4310483,"POLYGON ((-170630.251 2055.510, -170625.599 20...",2000
393,60830017041,963a355b784666,"MULTIPOLYGON (((48870.831 -403067.766, 48879.9...",2000


In [132]:
df_bg_intersect_geoall = df_bg_intersect_geoall.set_geometry('bg_int_geom')
df_bg_intersect_geoall.to_file('data/df_bg_geoall.shp', driver='ESRI Shapefile')

/Users/natebender/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  


# Aggregate by Flare ID and export files for Tableau

Take baseline dfs and create copies clearly defined for this flare buffer analysis, 
separate from the work that is needed for the BG buffer analysis above. 
Both analyses modify these same two dfs in different ways; need to ensure I keep the calculations separated.
Plus, they'll go into Tableau as different dataframes anyway. 

In [70]:
# Use the baseline df as the starting point

df_baseline_all.sample()

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,bg_geom,index_right,BCM_avg,flare_id,flare_cate,2000_buff_geom,2000_buff_int_prop,2000_buff_int_area,2000_buff_tot_area,2000_bg_int_prop,2000_bg_int_area,2000_bg_int_geom,2000_ACSTOTPOP_bg_int_count,2000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,2000_bg_int_mnr_prop
1862,60133141061,Contra Costa County,1306,886,48.87,"564,181.76","POLYGON ((-170231.325 2898.877, -170212.656 28...",89.00,0.01,342ab9d4310483,upstream,"POLYGON ((-167588.809 339.717, -167598.440 143...",0.01,"151,357.49","12,546,193.96",0.28,"155,946.68","POLYGON ((-169532.927 2341.461, -169533.073 23...",360.99,244.90,0.68,0.68


In [71]:
# Define the variables to aggregate

# BCM_avg: No adjustments needed because bcm is already attached to the flare unit of analysis
# D_PM25_D: Create new var of avg PM25 score for all BGs in each buffer 
# ACSTOTPOP_bg_int_count: Create new var of total population in all intersections in each buffer

agg_dict = {'D_PM25_2': 'mean', f'{buffer_size}_ACSTOTPOP_bg_int_count': 'sum', f'{buffer_size}_MINORPOP_bg_int_count': 'sum'}

# set up df for flare-specific calcs
flare_intersect = df_baseline_all.copy()

# Group the block groups by flare ID and aggregrate the selected vars
flare_agg = df_baseline_all.groupby('flare_id').agg(agg_dict)

# Rename the output variables
flare_agg = flare_agg.rename(columns={'D_PM25_2': f'{buffer_size}_int_pm25_avg', 
                                      f'{buffer_size}_ACSTOTPOP_bg_int_count': f'{buffer_size}_ACSTOTPOP_bg_int_sum',
                                     f'{buffer_size}_MINORPOP_bg_int_count': f'{buffer_size}_MINORPOP_bg_int_sum'})

# Join the aggregated variables with the original columns
flare_intersect = flare_intersect.merge(flare_agg, on='flare_id')

# add a new column for proportion of minorities within the given buffer zone
flare_intersect[f'{buffer_size}_flare_minor_prop'] = (flare_intersect[f'{buffer_size}_MINORPOP_bg_int_sum'] / 
                                                      flare_intersect[f'{buffer_size}_ACSTOTPOP_bg_int_sum'])


In [72]:
flare_intersect.sample(3)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,bg_geom,index_right,BCM_avg,flare_id,flare_cate,2000_buff_geom,2000_buff_int_prop,2000_buff_int_area,2000_buff_tot_area,2000_bg_int_prop,2000_bg_int_area,2000_bg_int_geom,2000_ACSTOTPOP_bg_int_count,2000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,2000_bg_int_mnr_prop,2000_int_pm25_avg,2000_ACSTOTPOP_bg_int_sum,2000_MINORPOP_bg_int_sum,2000_flare_minor_prop
317,60290033073,Kern County,831,443,38.34,"935,782,038.54","POLYGON ((48055.583 -341950.432, 48063.342 -34...",63.00,0.00,4255f961340565,upstream,"POLYGON ((71631.745 -327371.154, 71622.115 -32...",0.18,"2,231,043.93","12,546,193.96",0.01,"13,647,317.83","MULTIPOLYGON (((94524.369 -329984.954, 94995.4...",12.12,6.46,0.53,0.53,52.14,95.43,79.38,0.83
27,60375758012,Los Angeles County,722,622,62.89,"46,335.45","POLYGON ((166692.058 -469098.755, 166749.866 -...",113.00,0.00,857477a0194312,down_oil,"POLYGON ((167197.511 -469609.028, 167187.881 -...",0.00,"46,335.45","12,546,193.96",1.00,"46,335.45","POLYGON ((166692.058 -469098.755, 166749.866 -...",722.00,622.00,0.86,0.86,69.01,"19,265.89","16,185.06",0.84
417,60133552013,Contra Costa County,2986,2590,43.14,"2,345,993.58","POLYGON ((-171955.907 882.033, -171886.376 881...",89.00,0.01,342ab9d4310483,upstream,"POLYGON ((-167588.809 339.717, -167598.440 143...",0.12,"1,542,311.36","12,546,193.96",0.67,"1,561,565.19","POLYGON ((-171321.421 1356.557, -171326.181 13...","1,987.57","1,723.98",0.87,0.87,51.24,"12,150.90","10,380.55",0.85


In [73]:
# Define the variables to standardize

varlist = ['BCM_avg', f'{buffer_size}_int_pm25_avg', f'{buffer_size}_ACSTOTPOP_bg_int_sum']  

# Create a StandardScaler object that will transform selected variables to have a mean of zero and 
# sd of 1.
scaler = StandardScaler()

# Fit and transform the selected variables
intersect_norm = scaler.fit_transform(flare_intersect[varlist])

# Create new variables with the standardized values
intersect_norm = pd.DataFrame(intersect_norm, columns=[var + '_n' for var in varlist])

# Concatenate the new variables with the original DataFrame
flare_intersect = pd.concat([flare_intersect, intersect_norm], axis=1)

In [74]:
type(flare_intersect)

geopandas.geodataframe.GeoDataFrame

In [75]:
flare_intersect.sample(3)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,bg_geom,index_right,BCM_avg,flare_id,flare_cate,2000_buff_geom,2000_buff_int_prop,2000_buff_int_area,2000_buff_tot_area,2000_bg_int_prop,2000_bg_int_area,2000_bg_int_geom,2000_ACSTOTPOP_bg_int_count,2000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,2000_bg_int_mnr_prop,2000_int_pm25_avg,2000_ACSTOTPOP_bg_int_sum,2000_MINORPOP_bg_int_sum,2000_flare_minor_prop,BCM_avg_n,2000_int_pm25_avg_n,2000_ACSTOTPOP_bg_int_sum_n
199,60590626434,Orange County,594,252,19.52,"2,118,889.96","POLYGON ((201605.690 -485634.058, 201682.676 -...",88.00,0.00,7f8095b7782755,upstream,"POLYGON ((204125.440 -486756.777, 204115.810 -...",0.15,"1,849,124.98","12,546,193.96",0.87,"1,850,983.57","POLYGON ((201605.690 -485634.058, 201682.676 -...",518.90,220.14,0.42,0.42,25.34,"7,614.00","3,092.89",0.41,-0.56,-0.98,-0.26
47,60379800331,Los Angeles County,14,14,94.00,"28,637,875.64","POLYGON ((162259.677 -471740.729, 162351.365 -...",106.00,0.00,b30e8627682313,down_oil,"POLYGON ((166954.446 -469593.965, 166944.816 -...",0.54,"6,776,933.35","12,546,193.96",0.24,"6,924,234.07","POLYGON ((163103.041 -470349.141, 163435.050 -...",3.39,3.39,1.00,1.00,70.45,"17,414.85","15,000.86",0.86,-0.18,1.26,0.70
67,60375759013,Los Angeles County,557,497,67.03,"59,144.38","POLYGON ((166488.172 -469932.814, 166495.826 -...",107.00,0.00,caf4a94c957987,down_oil,"POLYGON ((167134.921 -469587.135, 167125.291 -...",0.00,"59,144.38","12,546,193.96",1.00,"59,144.38","POLYGON ((166488.172 -469932.814, 166495.826 -...",557.00,497.00,0.89,0.89,69.01,"19,265.89","16,185.06",0.84,-0.07,1.19,0.88


### 100

In [43]:
# Create a new geodataframe with just the id and geom columns
# flare_intersect_geo = flare_intersect[['flare_id', 'buff2000_geom']].copy()
flareintersect_geo100 = gp.GeoDataFrame(flare_intersect[['flare_id', '100_buff_geom']].copy(), 
                                      geometry='100_buff_geom', crs=meters_crs)


# Subset the original dataframe and drop unnecessary columns
flareintersect_100 = flare_intersect.drop(['bg_geom', '100_bg_int_geom', '100_buff_geom'], axis=1)

# Add in a buffer_size col for filtering in Tableau
flareintersect_geo100['buffer_size'] = 100

# and change the geom col name back to a general name so the 
# three buffer sizes can now be merged into one column
flareintersect_geo100.rename(columns={'100_buff_geom':'buff_geom'}, inplace=True)  # old:new.

In [59]:
flareintersect_100.sample(1)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,index_right,BCM_avg,flare_id,flare_cate,100_buff_int_prop,100_buff_int_area,100_buff_tot_area,100_bg_int_prop,100_bg_int_area,100_ACSTOTPOP_bg_int_count,100_MINORPOP_bg_int_count,MINORPOP_bg_totprop,100_bg_int_mnr_prop,100_int_pm25_avg,100_ACSTOTPOP_bg_int_sum,100_MINORPOP_bg_int_sum,100_flare_minor_prop,BCM_avg_n,100_int_pm25_avg_n,100_ACSTOTPOP_bg_int_sum_n
3,61110003021,Ventura County,1922,1570,31.02,"38,180,504.02",33.00,0.00,80ca7d95558230,upstream,1.00,"31,365.48","31,365.48",0.00,"64,580.35",3.25,2.66,0.82,0.82,31.02,3.25,2.66,0.82,-0.47,-0.24,0.13


### 1000

In [60]:
# Create a new geodataframe with just the id and geom columns
# flare_intersect_geo = flare_intersect[['flare_id', 'buff2000_geom']].copy()
flareintersect_geo1000 = gp.GeoDataFrame(flare_intersect[['flare_id', '1000_buff_geom']].copy(), 
                                      geometry='1000_buff_geom', crs=meters_crs)


# Subset the original dataframe and drop unnecessary columns
flareintersect_1000 = flare_intersect.drop(['bg_geom', '1000_bg_int_geom', '1000_buff_geom'], axis=1)

# Add in a buffer_size col for filtering in Tableau
flareintersect_geo1000['buffer_size'] = 1000

# and change the geom col name back to a general name so the 
# three buffer sizes can now be merged into one column
flareintersect_geo1000.rename(columns={'1000_buff_geom':'buff_geom'}, inplace=True)  # old:new.

In [61]:
flareintersect_1000.sample(1)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,index_right,BCM_avg,flare_id,flare_cate,1000_buff_int_prop,1000_buff_int_area,1000_buff_tot_area,1000_bg_int_prop,1000_bg_int_area,1000_ACSTOTPOP_bg_int_count,1000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,1000_bg_int_mnr_prop,1000_int_pm25_avg,1000_ACSTOTPOP_bg_int_sum,1000_MINORPOP_bg_int_sum,1000_flare_minor_prop,BCM_avg_n,1000_int_pm25_avg_n,1000_ACSTOTPOP_bg_int_sum_n
168,60290033042,Kern County,1040,279,31.94,"1,051,991,808.27",58.00,0.00,9895e6d1701977,upstream,1.00,"3,136,548.49","3,136,548.49",0.02,"19,762,400.65",19.54,5.24,0.27,0.27,31.94,19.54,5.24,0.27,0.23,-0.37,-0.61


### 2000

In [76]:
# Create a new geodataframe with just the id and geom columns
# flare_intersect_geo = flare_intersect[['flare_id', 'buff2000_geom']].copy()
flareintersect_geo2000 = gp.GeoDataFrame(flare_intersect[['flare_id', '2000_buff_geom']].copy(), 
                                      geometry='2000_buff_geom', crs=meters_crs)


# Subset the original dataframe and drop unnecessary columns
flareintersect_2000 = flare_intersect.drop(['bg_geom', '2000_bg_int_geom', '2000_buff_geom'], axis=1)

# Add in a buffer_size col for filtering in Tableau
flareintersect_geo2000['buffer_size'] = 2000

# and change the geom col name back to a general name so the 
# three buffer sizes can now be merged into one column
flareintersect_geo2000.rename(columns={'2000_buff_geom':'buff_geom'}, inplace=True)  # old:new.

In [77]:
flareintersect_2000.sample(1)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,index_right,BCM_avg,flare_id,flare_cate,2000_buff_int_prop,2000_buff_int_area,2000_buff_tot_area,2000_bg_int_prop,2000_bg_int_area,2000_ACSTOTPOP_bg_int_count,2000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,2000_bg_int_mnr_prop,2000_int_pm25_avg,2000_ACSTOTPOP_bg_int_sum,2000_MINORPOP_bg_int_sum,2000_flare_minor_prop,BCM_avg_n,2000_int_pm25_avg_n,2000_ACSTOTPOP_bg_int_sum_n
148,60376030082,Los Angeles County,1837,1790,59.06,"304,979.13",105.00,0.00,4338c869142492,upstream,0.02,"264,575.79","12,546,193.96",0.87,"264,575.79","1,593.64","1,552.86",0.97,0.97,62.28,"34,358.37","33,447.38",0.97,-0.54,0.86,2.35


### Combine

#### Geo data

In [482]:
flareintersect_geo100.sample(1)

,flare_id,buff_geom,buffer_size
2,e4c4a3a5502407,"POLYGON ((104173.072 -404640.555, 104172.590 -...",100


In [467]:
flareintersect_geo1000.sample(1)

,flare_id,buff_geom,buffer_size
129,bc2c1c15843944,"POLYGON ((68393.702 -411170.653, 68388.887 -41...",1000


In [451]:
flareintersect_geo2000.sample(1)

,flare_id,buff_geom,buffer_size
447,bd501374509776,"POLYGON ((82316.731 -428173.100, 82307.101 -42...",2000


In [483]:
df_flareintersect_geoall = pd.concat([flareintersect_geo100, 
                                    flareintersect_geo1000,
                                    flareintersect_geo2000])

In [485]:
df_flareintersect_geoall.sample(10)

,flare_id,buff_geom,buffer_size
426,47291ae3406417,"POLYGON ((-167598.970 344.705, -167608.600 148...",2000
362,a45c6de3182757,"POLYGON ((96141.089 -332289.622, 96131.459 -33...",2000
216,9f10973b328919,"POLYGON ((204129.924 -486745.806, 204120.294 -...",2000
35,857477a0194312,"POLYGON ((167197.511 -469609.028, 167187.881 -...",2000
212,0f78932c848460,"POLYGON ((102739.009 -392441.506, 102734.194 -...",1000
484,dc2822a7141447,"POLYGON ((11642.131 -145989.743, 11632.500 -14...",2000
99,719606ce832657,"POLYGON ((50145.686 -404732.075, 50145.205 -40...",100
28,4338c869142492,"POLYGON ((160089.293 -455288.160, 160084.478 -...",1000
81,36ecd8cd107104,"POLYGON ((95072.137 -332054.081, 95067.321 -33...",1000
392,f32d2397880993,"POLYGON ((-10811.858 -176053.069, -10821.488 -...",2000


In [494]:
df_flareintersect_geoall = df_flareintersect_geoall.set_geometry('buff_geom')
df_flareintersect_geoall.to_file('data/df_flare_geoall.shp', driver='ESRI Shapefile')

/Users/natebender/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  


#### Social data

In [78]:
def filter_dataframe(df, prefix):
    cols_to_keep = ['bg_id', 'flare_id'] + [col for col in df.columns if col.startswith(prefix)]
    return df[cols_to_keep].copy()

In [79]:
# filter the 2nd and 3rd dataframes to only include columns starting with the buffer cols
# eg '5000_' plus the ID columns
# all the non "5000_" columns in these dfs are identical to the first df, so can be ignored.
# select columns that start with the appropriate prefix and keep 'bg_id' and 'flare_id'

flareintersect_100_filtered = filter_dataframe(flareintersect_100, '100_')
flareintersect_1000_filtered = filter_dataframe(flareintersect_1000, '1000_')

In [80]:
df_flare_intersect_temp = flareintersect_2000.merge(flareintersect_1000_filtered, 
                                               on=['bg_id', 'flare_id'], how='outer')


df_flare_intersect_all = df_flare_intersect_temp.merge(flareintersect_100_filtered, 
                                               on=['bg_id', 'flare_id'], how='outer')

In [81]:
df_flare_intersect_all.to_csv('data/df_flare_impact_all.tsv', sep='\t', index=False)

## Folium Mapping

In [ ]:
# for_map = df_final[['flare_id', 'BCM_avg', 'D_PM25_2', 'ACSTOTPOP_intersect_count', 'impact_metric', 'buffer_2000m']]
# for_map = gp.GeoDataFrame(for_map, geometry='buffer_2000m', crs=meters_crs)

# for_map.to_file("data/df_impactmetric_shp.shp", driver='ESRI Shapefile')

In [ ]:

# # Define the color scale and number of bins
# #color_scale = 'Reds'
# #num_bins = 10

# # Create a map centered on the first flare
# # Create a folium map with a center location
# m = folium.Map(location=[38.377158,-121.645792], zoom_start=6, tiles=None,overlay=False)  #start w lat/long roughly in center of CA
# base_map = folium.FeatureGroup(name='Base map', overlay=True, control=False)
# folium.TileLayer(tiles='OpenStreetMap').add_to(base_map)
# base_map.add_to(m)




In [ ]:
# # Feature Group: Buffers

# def style_function3(feature):
#     return {
#         'fillColor': 'red',
#         'color': 'red',
#         'fillOpacity': 0.05
#     }

# inter_all_bg = folium.FeatureGroup(name='Intersect BG', overlay=True)
# folium.GeoJson(
#     data=test["buffer_2000m"],
#     style_function=style_function3
# ).add_to(inter_all_bg)
# inter_all_bg.add_to(m)

In [ ]:
# # Feature Group: block group

# def style_function2(feature):
#     return {
#         'fillColor': 'blue',
#         'color': 'blue',
#         'fillOpacity': 0.05
#     }

# bg_bound = folium.FeatureGroup(name='Intersect BG', overlay=True)
# folium.GeoJson(
#     data=test["bg_geom"],
#     style_function=style_function2
# ).add_to(bg_bound)
# bg_bound.add_to(m)

In [ ]:
# style_function = lambda x: {'fillColor': '#ffffff', 
#                             'color':'#000000', 
#                             'fillOpacity': 0.1, 
#                             'weight': 0.1}
# highlight_function = lambda x: {'fillColor': '#999999', 
#                                 'color':'#999999', 
#                                 'fillOpacity': 0.50, 
#                                 'weight': 0.1}
# NIL = folium.features.GeoJson(
#     data = for_map,
#     style_function=style_function, 
#     control=False,
#     highlight_function=highlight_function, 
#     tooltip=folium.features.GeoJsonTooltip(
#         fields=['flare_id', 'BCM_avg_norm', 'D_PM25_2_norm', 
#              'ACSTOTPOP_intersect_count_norm', 'impact_metric'],# 'D_PM25_2', 'ACSTOTPOP', 'MINORPOP','shape_area_new', 'intersect_prop', 'intersect_area', 'MINORPOP_bg_totprop'],
#         style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
#     )
# )

In [ ]:
# # add hover functionality as child to map, add layering, display map
# m.add_child(NIL)
# m.keep_in_front(NIL)
# folium.LayerControl().add_to(m)

In [ ]:
# # Display the map
# m